In [1]:
with open('input000.txt', 'r') as f:
    content = f.readlines()

In [2]:
content.pop(0)

'250\n'

In [178]:
content

['1/3/2012 16:00:00\t26.96\n',
 '1/4/2012 16:00:00\t27.47\n',
 '1/5/2012 16:00:00\t27.728\n',
 '1/6/2012 16:00:00\t28.19\n',
 '1/9/2012 16:00:00\t28.1\n',
 '1/10/2012 16:00:00\t28.15\n',
 '1/11/2012 16:00:00\t27.98\n',
 '1/12/2012 16:00:00\t28.02\n',
 '1/13/2012 16:00:00\t28.25\n',
 '1/17/2012 16:00:00\t28.65\n',
 '1/18/2012 16:00:00\t28.4\n',
 '1/19/2012 16:00:00\t28.435\n',
 '1/20/2012 16:00:00\t29.74\n',
 '1/23/2012 16:00:00\t29.95\n',
 '1/24/2012 16:00:00\t29.5703\n',
 '1/25/2012 16:00:00\t29.65\n',
 '1/26/2012 16:00:00\t29.7\n',
 '1/27/2012 16:00:00\t29.53\n',
 '1/30/2012 16:00:00\t29.62\n',
 '1/31/2012 16:00:00\t29.7\n',
 '2/1/2012 16:00:00\t30.05\n',
 '2/2/2012 16:00:00\t30.17\n',
 '2/3/2012 16:00:00\t30.4\n',
 '2/6/2012 16:00:00\t30.22\n',
 '2/7/2012 16:00:00\t30.485\n',
 '2/8/2012 16:00:00\t30.67\n',
 '2/9/2012 16:00:00\t30.8\n',
 '2/10/2012 16:00:00\t30.8\n',
 '2/13/2012 16:00:00\t30.77\n',
 '2/14/2012 16:00:00\t30.46\n',
 '2/15/2012 16:00:00\t30.39\n',
 '2/16/2012 16:00:00\t

In [3]:
#Import
from datetime import datetime
from dateutil.parser import parse
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import xgboost as xgb
from sklearn import utils
import pandas as pd
import numpy as np

In [55]:
merc = []
time = []

for reading in content:
    float_days = datetime.strptime(reading.split("\t")[0], '%m/%d/%Y %H:%M:%S')
    time.append(float_days)
    try:
        merc.append(float(reading.split("\t")[1]))
    except:
        merc.append(np.nan)
        pass

In [56]:
df = pd.Series(merc, index=time)
df.index.name = 'Date'
df.head()

Date
2012-01-03 16:00:00    26.960
2012-01-04 16:00:00    27.470
2012-01-05 16:00:00    27.728
2012-01-06 16:00:00    28.190
2012-01-09 16:00:00    28.100
dtype: float64

In [50]:
df = df.reset_index(name='Reading')
df.head()

,Date,Reading
0,2012-01-03 16:00:00,26.960
1,2012-01-04 16:00:00,27.470
2,2012-01-05 16:00:00,27.728
3,2012-01-06 16:00:00,28.190
4,2012-01-09 16:00:00,28.100


In [7]:
missing_reading = df[df['Reading'].isnull()]['Date'].values
missing_reading = missing_reading.astype('datetime64[D]').astype(int)
missing_reading = [[x] for x in missing_reading]
missing_reading = np.asarray(missing_reading)
missing_reading

array([[15412],
       [15420],
       [15425],
       [15484],
       [15492],
       [15495],
       [15541],
       [15544],
       [15547],
       [15593],
       [15600],
       [15603],
       [15622],
       [15623],
       [15628],
       [15644],
       [15650],
       [15657],
       [15660],
       [15687]])

In [8]:
df = df.dropna()
dates, reads = [[x] for x in df['Date'].values], df['Reading'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(dates, reads, test_size=0.05, shuffle=False)
X_train, y_train = np.asarray(X_train), np.asarray(y_train)

In [10]:
mdl = RandomForestRegressor(n_estimators=1000, max_depth=500, random_state=0)
mdl.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=500,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [11]:
y_pred = mdl.predict(missing_reading)
for pred in y_pred:
    print(pred)

27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617
27.202471999999617


## Machine Learning

In [12]:
target = df['Reading'].values.astype(int)

In [13]:
dates1 = df['Date'].values
dates1 = dates1.astype('datetime64[D]').astype(int)
dates1 = [[x] for x in dates1]
dates1 = np.array(dates1)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(dates1, target, test_size=0.2)

### XGBoost

In [15]:
#Instantiate the Classifier
xgboost = xgb.XGBClassifier()

In [16]:
#Create a Grid for GridsearchCV to find the best parameters for the classifier
xg_param_grid = {'n_estimators': [200],
              'learning_rate': [0.05, 0.1], 
              'max_depth': [6,7],
              'colsample_bytree': [0.7, 1],
              'gamma': [0.0, 0.1, 0.2]}

xg_search = GridSearchCV(xgboost, xg_param_grid, scoring='accuracy', cv=3)

xg_search.fit(X_train, y_train)

print(xg_search.best_params_)

KeyboardInterrupt: 

In [17]:
# Instantiate the XGBoost classifier using GridSearchCV hyperparameters
booster = xgb.XGBClassifier(learning_rate=0.1, max_depth=7, n_estimators=200,
                            colsample_bytree=0.7, gamma=0.1, random_state=42)

In [18]:
#Train the data on the model
booster.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [19]:
pred = booster.predict(missing_reading)

In [21]:
for x in np.nditer(pred): 
   print(x,)

32
32
32
29
29
29
30
29
29
31
31
31
29
29
29
29
29
27
26
27


### RFC

In [22]:
#Instantiate the RandomForestClassifier for GridsearchCV
forest = RandomForestClassifier()

In [211]:
#Create a Grid for GridsearchCV to find the best parameters for the classifier
rf_param_grid = {'n_estimators': [100,250,500,1000],
                'criterion': ['gini', 'entropy'],
                'max_depth': [None, 2, 6, 10],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [2, 3, 6]}

gs_tree = GridSearchCV(forest, rf_param_grid, cv=3)

gs_tree.fit(X_train, y_train)
print(gs_tree.best_params_)

C:\Users\John\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 1000}


In [23]:
forest = RandomForestClassifier(n_estimators=1000, criterion='entropy', 
                                max_depth=6, min_samples_split=2, min_samples_leaf=2, random_state=42)

In [24]:
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=6, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [25]:
pred2 = forest.predict(missing_reading)

In [26]:
for x in np.nditer(pred2): 
   print(x,)

32
32
32
29
29
29
29
29
29
31
31
31
29
29
29
29
29
29
26
27


In [27]:
for x in np.nditer(pred2): 
   print(x,)

32
32
32
29
29
29
29
29
29
31
31
31
29
29
29
29
29
29
26
27


In [28]:
linreg = LinearRegression()

In [29]:
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [30]:
lin_pred = linreg.predict(missing_reading)

In [31]:
lin_pred

array([30.47157605, 30.40503293, 30.36344348, 29.87268795, 29.80614483,
       29.78119116, 29.39856821, 29.37361454, 29.34866087, 28.96603791,
       28.90781268, 28.88285901, 28.72481909, 28.7165012 , 28.67491175,
       28.54182551, 28.49191816, 28.43369293, 28.40873926, 28.18415622])

In [32]:
for x in np.nditer(lin_pred): 
   print(x,)

30.471576054492118
30.405032932155393
30.36344348069494
29.872687953461508
29.806144831124755
29.781191160248483
29.398568206812257
29.373614535935957
29.348660865059685
28.96603791162346
28.907812679578797
28.882859008702525
28.724819093152774
28.716501202860684
28.67491175140023
28.541825506726752
28.49191816497421
28.433692932929546
28.408739262053274
28.1841562241668


## Pandas Interpolation

In [155]:
merc = []
time = []

for reading in content:
    float_days = datetime.strptime(reading.split("\t")[0], '%m/%d/%Y %H:%M:%S')
    time.append(float_days)
    try:
        merc.append(float(reading.split("\t")[1]))
    except:
        merc.append(np.nan)
        pass

In [156]:
df = pd.Series(merc, index=time)
df.index.name = 'Date'
df.head()

Date
2012-01-03 16:00:00    26.960
2012-01-04 16:00:00    27.470
2012-01-05 16:00:00    27.728
2012-01-06 16:00:00    28.190
2012-01-09 16:00:00    28.100
dtype: float64

In [158]:
df = df.reset_index(name='Reading')
missing_reading = df.loc[df.isnull().any(axis=1)]
df.head()

,Date,Reading
0,2012-01-03 16:00:00,26.960
1,2012-01-04 16:00:00,27.470
2,2012-01-05 16:00:00,27.728
3,2012-01-06 16:00:00,28.190
4,2012-01-09 16:00:00,28.100


In [171]:
rows = []
for index, row in missing_reading.iterrows():
    rows.append(index)

In [159]:
missing_reading

,Date,Reading
48,2012-03-13 16:00:00,NaN
54,2012-03-21 16:00:00,NaN
57,2012-03-26 16:00:00,NaN
99,2012-05-24 16:00:00,NaN
104,2012-06-01 16:00:00,NaN
105,2012-06-04 16:00:00,NaN
138,2012-07-20 16:00:00,NaN
139,2012-07-23 16:00:00,NaN
142,2012-07-26 16:00:00,NaN
173,2012-09-10 16:00:00,NaN


In [160]:
df.Date = pd.to_datetime(df.Date, format='%d-%m-%Y')
df = df.set_index('Date')
df.head()

,Reading
Date,
2012-01-03 16:00:00,26.960
2012-01-04 16:00:00,27.470
2012-01-05 16:00:00,27.728
2012-01-06 16:00:00,28.190
2012-01-09 16:00:00,28.100


In [161]:
df = df.assign(Reading=df.Reading.interpolate(method='time'))

In [162]:
df.head()

,Reading
Date,
2012-01-03 16:00:00,26.960
2012-01-04 16:00:00,27.470
2012-01-05 16:00:00,27.728
2012-01-06 16:00:00,28.190
2012-01-09 16:00:00,28.100


In [166]:
df.iloc[48]

Reading    32.54
Name: 2012-03-13 16:00:00, dtype: float64

In [163]:
missing_reading

,Date,Reading
48,2012-03-13 16:00:00,NaN
54,2012-03-21 16:00:00,NaN
57,2012-03-26 16:00:00,NaN
99,2012-05-24 16:00:00,NaN
104,2012-06-01 16:00:00,NaN
105,2012-06-04 16:00:00,NaN
138,2012-07-20 16:00:00,NaN
139,2012-07-23 16:00:00,NaN
142,2012-07-26 16:00:00,NaN
173,2012-09-10 16:00:00,NaN


In [167]:
df.head()

,Reading
Date,
2012-01-03 16:00:00,26.960
2012-01-04 16:00:00,27.470
2012-01-05 16:00:00,27.728
2012-01-06 16:00:00,28.190
2012-01-09 16:00:00,28.100


In [177]:
for i in rows:
    print(df.iloc[i].Reading)

32.540000000000006
32.120000000000005
32.5525
29.38
29.286
28.884
30.512
29.648
29.59
30.95
31.22
31.4
29.69666666666667
29.473333333333333
29.634999999999998
29.356666666666666
29.7825
28.15
26.9275
27.375
